<a href="https://colab.research.google.com/github/duncansamuelgeorgefreeman/colab/blob/master/AppliedNLPML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy
!pip install -U spacy-lookups-data
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8MB 4.0MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
     |████████████████████████████████| 9.2MB 45.1MB/s 
     |████████████████████████████████| 460kB 51.3MB/s 
     |████████████████████████████████| 122kB 41.3MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=53088fe85fff9884e5434970e5211cb21a3f276307d60bc8347e282974487aff
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for smart-open: filename=smart_open-3.0.0-cp36-none-any.whl size=107097 sha256=eec5520782c4c8d02a8dd6bf27efe3de09549871df62a4444d620d3671f0e828
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built contextvars smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Suc

In [2]:
# download core spacy language model
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
# tokenization - isolate tokenizer from spacy nlp pipeline and tokenize text.
sentence = nlp.tokenizer("I live in Pittsburgh")
print(len(sentence))
for w in sentence:
  print(w)

4
I
live
in
Pittsburgh


In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
! ls '/gdrive/My Drive/dev_colab/nlp_b/'

jeopardy.csv


In [6]:
# import Jeopardy questions dataset
import pandas as pd

data = pd.read_csv('/gdrive/My Drive/dev_colab/nlp_b/jeopardy.csv')
df = pd.DataFrame(data=data)
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [7]:
df.columns = map(lambda x: x.lower().strip().replace(" ", "_"), df.columns)
df.columns

Index(['show_number', 'air_date', 'round', 'category', 'value', 'question',
       'answer'],
      dtype='object')

In [8]:
# shrink dataset and create a field for tokenized Jeopardy questions.
df = df[:1000]
df['question_tokenized'] = df['question'].apply(lambda x: nlp(x))
df.head(2)

,show_number,air_date,round,category,value,question,answer,question_tokenized
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,"(For, the, last, 8, years, of, his, life, ,, G..."
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,"(No, ., 2, :, 1912, Olympian, ;, football, sta..."


In [9]:
# POS tagging: use spacy pos tagging attribute.
pos_tags = {"ADJ":["adjective", "big, old, green, incomprehensible, first"], 
            "ADP":["adposition", "in, to, during"], 
            "ADV":["adverb", "very, tomorrow, down, where, there"], 
            "AUX":["auxiliary", "is, has (done), will (do), should (do)"], 
            "CONJ": ["conjunction", "and, or, but"], 
            "CCONJ": ["coordinating conjunction", "and, or, but"], 
            "DET": ["determiner", "a, an, the"], 
            "INTJ": ["interjection", "psst, ouch, bravo, hello"], 
            "NOUN": ["noun", "girl, cat, air, tree, beauty"], 
            "NUM": ["numeral", "1, 2021, one, IVXIX"], 
            "PART": ["particle", "'s, not"], 
            "PRON": ["pronoun", "I, you, she, he, myself, themselves, somebody"], 
            "PROPN": ["proper noun", "Duncan, London, HBO"], 
            "PUNCT": ["punctuation", ".,;()?"], 
            "SCONJ":["subordinating conjunction", "if, while, that"], 
            "SYM":["symbol", "$%@+=:)"], 
            "VERB":["verb", "run, runs, eat, ate, eating"], "X": ["other", "sfghjkldfg"], "SPACE": ["space", " "]}
for token in df['question_tokenized'][0]:
  print(f"{token.text}\t{token.pos_}\t{pos_tags[token.pos_][0]}\t{pos_tags[token.pos_][1]}")
  #print(token.text, token.pos_, pos_tags[token.pos_][0], pos_tags[token.pos_][1])

For	ADP	adposition	in, to, during
the	DET	determiner	a, an, the
last	ADJ	adjective	big, old, green, incomprehensible, first
8	NUM	numeral	1, 2021, one, IVXIX
years	NOUN	noun	girl, cat, air, tree, beauty
of	ADP	adposition	in, to, during
his	PRON	pronoun	I, you, she, he, myself, themselves, somebody
life	NOUN	noun	girl, cat, air, tree, beauty
,	PUNCT	punctuation	.,;()?
Galileo	PROPN	proper noun	Duncan, London, HBO
was	AUX	auxiliary	is, has (done), will (do), should (do)
under	ADP	adposition	in, to, during
house	NOUN	noun	girl, cat, air, tree, beauty
arrest	NOUN	noun	girl, cat, air, tree, beauty
for	ADP	adposition	in, to, during
espousing	VERB	verb	run, runs, eat, ate, eating
this	DET	determiner	a, an, the
man	NOUN	noun	girl, cat, air, tree, beauty
's	PART	particle	's, not
theory	NOUN	noun	girl, cat, air, tree, beauty


In [10]:
# DEPENDENCY PARSING: A TOKEN'S RELATIONSHIP TO OTHER TOKENS/
# Dependency parsing: use tokens and POS data to label token's relationships to other tokens (find inherent structure among tokens).
# print dependency parsing tags for each token in question.
for token in df['question_tokenized'][0]:
  print(token.text, token.dep_)

For prep
the det
last amod
8 nummod
years pobj
of prep
his poss
life pobj
, punct
Galileo nsubj
was ROOT
under prep
house compound
arrest pobj
for prep
espousing pcomp
this det
man poss
's case
theory dobj


In [11]:
# visualize dependency parse.
from spacy import displacy
displacy.render(df['question_tokenized'][0], style='dep', jupyter=True, options={'distance':120})

In [12]:
# chunking (combine related tokens into single token). Grouping related tokens into chunks makes it easier for the machine to process the sentence.
## Instead of viewing each token in isolation, machine now recognizes certain tokens are related to others.
for chunk in nlp("I visited New York City and went to The Empire State Building.").noun_chunks:
  print(chunk.text)

I
New York City
The Empire State Building


In [13]:
# Lemmatization (convert word into base (canonical) form). Simplifies tokens by converting to most basic form.
lemma_df = pd.DataFrame(data=[], columns=["original_token", "lemmatized_token"])
for (i, token) in enumerate(df['question_tokenized'][0]):
  lemma_df.loc[i, "original_token"] = token.text
  lemma_df.loc[i, "lemmatized_token"] = token.lemma_

lemma_df

,original_token,lemmatized_token
0,For,for
1,the,the
2,last,last
3,8,8
4,years,year
5,of,of
6,his,his
7,life,life
8,",",","
9,Galileo,Galileo


In [14]:
# NER
sentence_ex = "George Washington was an American political leader, military general, statesman, and Founding Father who was the first president of the United States from 1789 to 1797."
for token in nlp(sentence_ex).ents:
  print(f"Text: {token.text},  Start: {token.start_char},  End: {token.end_char},  Label: {token.label_}")

Text: George Washington,  Start: 0,  End: 17,  Label: PERSON
Text: American,  Start: 25,  End: 33,  Label: NORP
Text: first,  Start: 113,  End: 118,  Label: ORDINAL
Text: the United States,  Start: 132,  End: 149,  Label: GPE
Text: 1789 to 1797,  Start: 155,  End: 167,  Label: DATE


In [15]:
# Use spacy entity recognizer visualizer to view text labels.
sentence_ex_nlp = nlp(sentence_ex)
displacy.render(sentence_ex_nlp, style='ent', jupyter=True)

In [16]:
# Named Entity Linking: Resolves a textual entity to a unique identifier in a knowledge base (source text to canonical version in knowledge db).
## EXAMPLE: Link all entities named "PERSON"'s to the Google Knowledge Graph.
#import requests
from __future__ import print_function
import requests
import json

# Define a Google knowledge graph graph api result function:
def get_graph_result(query, api_key, entity_type):
  if entity_type == "PERSON":
    request_str = f"https://kgsearch.googleapis.com/v1/entities:search?query={query}&key={api_key}"
    api_response = requests.get(request_str)
    url = api_response.json()['itemListElement'][0]['result']['detailedDescription']['url']
    description = api_response.json()['itemListElement'][0]['result']['detailedDescription']['articleBody']
    return url, description
  else:
    return "no_match", "no_match"

In [19]:
API_KEY = open('/gdrive/My Drive/dev_colab/nlp_b/keys/google_kg_apikey.txt').read()
# Perform entity linking on sentence example:
for token in nlp(sentence_ex).ents:
  url, description = get_graph_result(query=token.text, api_key=API_KEY, entity_type=token.label_)
  print(token.text, token.label_, description, url)

George Washington PERSON The George Washington University is a private research university in Washington, D.C. It was chartered in 1821 by an act of the United States Congress.
 https://en.wikipedia.org/wiki/George_Washington_University
American NORP no_match no_match
first ORDINAL no_match no_match
the United States GPE no_match no_match
1789 to 1797 DATE no_match no_match
